In [11]:
from keras.models import load_model
from keras.preprocessing import sequence
import numpy as np
import jieba.analyse
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary

model = load_model('weight.h5')

In [4]:
# 文本处理
def deal_text():
    data = open('data.txt').readlines()
    X=[]
    Y_=[]
    for i in data:
        text = i.split('***',1)
        X.append(text[0])
        Y_.append(text[1].replace('\n',''))
    Y_value = list(set(Y_))
    class_data = {i:Y_value.index(i) for i in Y_value}
    sentences_list = []
    for line in X:
        single_list = line.strip().split(' ')
        single_list = jieba.analyse.extract_tags(single_list[0],topK=20,withWeight=False,allowPOS=())
        while '' in single_list:
            single_list.remove('')
        sentences_list.append(single_list)
    return class_data,sentences_list

In [5]:
def create_dictionaries(model):
    gensim_dict = Dictionary()
    gensim_dict.doc2bow(model.wv.vocab.keys(),allow_update=True)
    w2indx = {v:k+1 for k,v in gensim_dict.items()}
    w2vec = {word:model[word] for word in w2indx.keys()}
    return w2indx,w2vec

In [6]:
# 在字典中找到词返回索引
# 文本和词典匹配将我们的词特征转为数字
def text_to_index_array(dic,sentence):
    new_sentence = []
    for sen in sentence:
        new_sen = []
        for word in sen:
            try:
                new_sen.append(dic[word])
            except:
                new_sen.append(0)
        new_sentence.append(new_sen)
    return np.array(new_sentence)

In [13]:
def convert_vector_predict(str_r,index_dict):
    new_str = jieba.analyse.extract_tags(str_r,topK=20,withWeight=False,allowPOS=())
    x = text_to_index_array(index_dict,[new_str])
    x = sequence.pad_sequences(x,maxlen=50)
    y = model.predict_proba(x)
    return y

In [14]:
class_data,sentences_list = deal_text()
model_vec = Word2Vec(sentences_list,size=100,min_count=5,window=5)
index_dict,word_vectors = create_dictionaries(model_vec)
value = convert_vector_predict('我用360借条一次性还清了所有欠款！',index_dict)
# print([k for k,v in class_data.items() if v==value[0]][0])
print(value)

[[4.51283813e-01 9.88334185e-04 1.14164184e-04 7.02058300e-02
  5.49979368e-03 2.28769728e-03 1.94052551e-02 3.18543106e-01
  5.45779839e-02 1.57773383e-02 4.43062261e-02 1.70105305e-02]]


/home/xms/.virtualenvs/dl/lib/python3.5/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
